In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split

from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

### Part 1: Validation set approach

In [3]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
print(f'Validation MSE: {np.mean((y_valid - valid_pred)**2)}')

Validation MSE: 23.61661706966988


In [4]:
def evalMSE(terms, response, train, test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test- test_pred)**2)

MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

In [5]:
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=3)

MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

### Part 2: Cross-validation

In [6]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))

X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0])

cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792922

In [13]:
validation = ShuffleSplit(n_splits=1, test_size=196, random_state=0)
results = cross_validate(hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv=validation)
print(f"Using 1 split: {results['test_score'][0]}")

Using 1 split: 23.61661706966988


In [12]:
validation = ShuffleSplit(n_splits=10, test_size=196, random_state=0)
results = cross_validate(hp_model, Auto.drop(['mpg'], axis=1), Auto['mpg'], cv=validation)
print(f"Using 10 splits: mean error {results['test_score'].mean()} and variance {results['test_score'].std()}")

Using 10 splits: mean error 23.802232661034168 and variance 1.4218450941091842


### Part 3: Bootstrap approach

In [16]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1]- cov_[0,1]) / (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

print(f'Estimated variance by all observations: {alpha_func(Portfolio, range(100))}')

Estimated variance by all observations: 0.57583207459283


In [26]:
rng = np.random.default_rng(0)
print(f'Estimated variance by bootstrap: {alpha_func(Portfolio, rng.choice(100, 100, replace=True))}')

Estimated variance by bootstrap: 0.6074452469619004


In [20]:
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B- (first_ / B)**2)

alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)
print(f'Standard error of alpha_func by bootstrap: {alpha_SE}')

Standard error of alpha_func by bootstrap: 0.09118176521277699


In [22]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')
rng = np.random.default_rng(0)

hp_se = boot_SE(hp_func, Auto, B=1000, seed=10)
print(f'Standard error for intercept: {hp_se.values[0]}')
print(f'Standard error for horsepower: {hp_se.values[1]}')

Standard error for intercept: 0.7311764728742826
Standard error for horsepower: 0.006091893031262252


In [23]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [24]:
quad_model = MS([poly('horsepower',  2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

In [25]:
M = sm.OLS(Auto['mpg'], quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64